In [ ]:
import numpy as np 
import pandas as pd 



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip /content/drive/Shareddrives/Hackathon/AMOC/sentiment140.zip -d /content/
!mv /content/'training.1600000.processed.noemoticon.csv' /content/dataset.csv

Archive:  /content/drive/Shareddrives/Hackathon/AMOC/sentiment140.zip
  inflating: /content/training.1600000.processed.noemoticon.csv  


# Loading the Dataset, We use engine=python as loading was giving error ( not able to read large sentences ) 

In [ ]:
data=pd.read_csv("/content/dataset.csv", encoding = "ISO-8859-1", engine="python")

In [ ]:
data.tail(5)

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
1599994,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599995,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599996,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599997,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...
1599998,4,2193602129,Tue Jun 16 08:40:50 PDT 2009,NO_QUERY,RyanTrevMorris,happy #charitytuesday @theNSPCC @SparksCharity...


In [ ]:


data.head(5)

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


# modifying data columns for better understanding

In [ ]:
data.columns = ["label", "time", "date", "query", "username", "text"]


In [ ]:
data

,label,time,date,query,username,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
...,...,...,...,...,...,...
1599994,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599995,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599996,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599997,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


# Checking if values are null


In [ ]:
data.isnull().sum()

label       0
time        0
date        0
query       0
username    0
text        0
dtype: int64

# Only text and label should be used as conclusion

In [ ]:
data=data[['label','text']]

In [ ]:
data['text']=data['text'].str.lower()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


# We use Nltk library for text analysis
Various procedures involved are :-

# i) Cleaning DATA


In [ ]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Importing stop words from nltk 

In [ ]:
data.label.value_counts()

4    800000
0    799999
Name: label, dtype: int64

# words used as stop words

In [ ]:
from nltk.corpus import stopwords
stopwords_list = stopwords.words('english')

FIRST 5 STOPWORDS

In [ ]:
stopwords_list[0:5]

['i', 'me', 'my', 'myself', 'we']

In [ ]:
data_pos = data[data['label'] == 4]
data_neg = data[data['label'] == 0]

In [ ]:
data_pos = data_pos.iloc[:int(40000)]
data_neg = data_neg.iloc[:int(40000)]

In [ ]:
data = pd.concat([data_pos, data_neg])

# Removing stopwords from dataset

In [ ]:
STOPWORDS = set(stopwords.words('english'))
def cleaning_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
data['text'] = data['text'].apply(lambda text: cleaning_stopwords(text))
data['text'].head()

799999                love @health4uandpets u guys r best!!
800000    im meeting one besties tonight! cant wait!! - ...
800001    @darealsunisakim thanks twitter add, sunisa! g...
800002    sick really cheap hurts much eat real food plu...
800003                      @lovesbrooklyn2 effect everyone
Name: text, dtype: object

# Cleaning and removing punctutions

In [ ]:
import string
english_punctuations = string.punctuation
punctuations_list = english_punctuations
punctuations_list[0:]

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
def cleaning_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
data['text']= data['text'].apply(lambda x: cleaning_punctuations(x))
data['text'].tail()

39995    mittec problems literally spent 2 hours cleani...
39996                                              ow head
39997    anyway particularly looking foreward labs toda...
39998                     allergies insomnia matter reason
39999    scottcabal im working evening could daytime su...
Name: text, dtype: object

# Cleaning and removing URL's

In [ ]:
import re
def cleaning_URLs(data):
    return re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',str(data))
data['text'] = data['text'].apply(lambda x: cleaning_URLs(x))
data['text'].tail()

39995    mittec problems literally spent 2 hours cleani...
39996                                              ow head
39997    anyway particularly looking foreward labs toda...
39998                     allergies insomnia matter reason
39999    scottcabal im working evening could daytime su...
Name: text, dtype: object

# Removing Numeric characters

In [ ]:
def cleaning_numbers(data):
    return re.sub('[0-9]+', '', data)
data['text'] = data['text'].apply(lambda x: cleaning_numbers(x))
data['text'].tail()

39995    mittec problems literally spent  hours cleanin...
39996                                              ow head
39997    anyway particularly looking foreward labs toda...
39998                     allergies insomnia matter reason
39999    scottcabal im working evening could daytime su...
Name: text, dtype: object

# ii) TOKENIZING 
Tokenization is the process of converting data into smaller sub parts for better analysis ( Eg: breaking words in sentence)

In [ ]:
from nltk.tokenize import RegexpTokenizer

In [ ]:
tokenizer = RegexpTokenizer(r'\w+')
data['text'] = data['text'].apply(tokenizer.tokenize)
data.head()

,label,text
799999,4,"[love, healthuandpets, u, guys, r, best]"
800000,4,"[im, meeting, one, besties, tonight, cant, wai..."
800001,4,"[darealsunisakim, thanks, twitter, add, sunisa..."
800002,4,"[sick, really, cheap, hurts, much, eat, real, ..."
800003,4,"[lovesbrooklyn, effect, everyone]"


# iii) Applying Stemming
We reduce words to their root form

In [ ]:
from nltk.stem import PorterStemmer 

In [ ]:
stemer = nltk.PorterStemmer()
def stemming_on_text(data):
    text = [stemer.stem(word) for word in data]
    return data

data['text']= data['text'].apply(lambda x: stemming_on_text(x))

In [ ]:
data.head()

,label,text
799999,4,"[love, healthuandpets, u, guys, r, best]"
800000,4,"[im, meeting, one, besties, tonight, cant, wai..."
800001,4,"[darealsunisakim, thanks, twitter, add, sunisa..."
800002,4,"[sick, really, cheap, hurts, much, eat, real, ..."
800003,4,"[lovesbrooklyn, effect, everyone]"


# iv) Applying Lemmatization
It is same as stemming but it also looks for dictionary relation while stemming is dependent on part of speech

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
lm = nltk.WordNetLemmatizer()
def lemmatizer_on_text(data):
    text = [lm.lemmatize(word) for word in data]
    return data

data['text'] = data['text'].apply(lambda x: lemmatizer_on_text(x))

In [ ]:
data.head()

,label,text
799999,4,"[love, healthuandpets, u, guys, r, best]"
800000,4,"[im, meeting, one, besties, tonight, cant, wai..."
800001,4,"[darealsunisakim, thanks, twitter, add, sunisa..."
800002,4,"[sick, really, cheap, hurts, much, eat, real, ..."
800003,4,"[lovesbrooklyn, effect, everyone]"


Now data cleaning is done

In [219]:
input=data['text']
output=data['label']
data['text']=data['text'].apply(lambda x: "4".join(x))


799999    l4 4o4 4v4 4e4 4 4 4h4 4e4 4a4 4l4 4t4 4h4 4u4...
800000    i4 4m4 4 4 4m4 4e4 4e4 4t4 4i4 4n4 4g4 4 4 4o4...
800001    d4 4a4 4r4 4e4 4a4 4l4 4s4 4u4 4n4 4i4 4s4 4a4...
800002    s4 4i4 4c4 4k4 4 4 4r4 4e4 4a4 4l4 4l4 4y4 4 4...
800003    l4 4o4 4v4 4e4 4s4 4b4 4r4 4o4 4o4 4k4 4l4 4y4...
                                ...                        
39995     m4 4i4 4t4 4t4 4e4 4c4 4 4 4p4 4r4 4o4 4b4 4l4...
39996                             o4 4w4 4 4 4h4 4e4 4a4 4d
39997     a4 4n4 4y4 4w4 4a4 4y4 4 4 4p4 4a4 4r4 4t4 4i4...
39998     a4 4l4 4l4 4e4 4r4 4g4 4i4 4e4 4s4 4 4 4i4 4n4...
39999     s4 4c4 4o4 4t4 4t4 4c4 4a4 4b4 4a4 4l4 4 4 4i4...
Name: text, Length: 80000, dtype: object

In [ ]:
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing import sequence
# max_len = 1000
# tok = Tokenizer(num_words=2000)
# tok.fit_on_texts(input)
# sequences = tok.texts_to_sequences(input)
# sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(sequences_matrix,output,test_size=0.3)

In [ ]:
model.fit(x_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
y_pred=model.predict(x_test)

In [ ]:
from sklearn.metrics import classification_report
cf=classification_report(y_test,y_pred)

In [ ]:
print(cf)

              precision    recall  f1-score   support

           0       0.53      0.28      0.37     12008
           4       0.51      0.75      0.61     11992

    accuracy                           0.52     24000
   macro avg       0.52      0.52      0.49     24000
weighted avg       0.52      0.52      0.49     24000

